In [1]:
import numpy as np
import pandas as pd

In [2]:
fldpnn_dict = {}
num_seqs = 0

for i in [1, 3, 4, 5, 6]:
    with open('../../data/fldpnn/fldpnn_scer-aa-filtered_{}.csv'.format(i)) as f:
        for line in f:
            if line.startswith('>'):
                if num_seqs != 0:
                    fldpnn_dict[name] = (new_seq, score_list, binary_list)
                new_seq = ''
                score_list = []
                binary_list = []
                name = line.split(' ')[0].strip('>')
                num_seqs += 1
            elif not line.startswith('R'):
                try:
                    elements = line.split(',')
                    new_seq += elements[1]
                    score_list.append(float(elements[2]))
                    binary_list.append(int(elements[3]))
                except:
                    print(i, elements)

5 ['418', 'E', '0.026']


In [27]:
def append_fldpnn_label(row):
    orf = row['orf']
    left_bound = row['left_bound']
    right_bound = row['right_bound']
    region = row['region_seq']
    if orf in fldpnn_dict:
        fldpnn_seq, fldpnn_score, fldpnn_binary  = fldpnn_dict[orf]
        fldpnn_region = fldpnn_seq[left_bound:(right_bound+1)]
        if not fldpnn_region == region:
            print(fldpnn_region)
            print(region)
        #assert fldpnn_region == region
        total_score = np.mean(fldpnn_score[left_bound:(right_bound+1)])
        if total_score >= 0.3:
            return 'disordered'
        else:
            return 'helix'
    else:
        return None

In [7]:
fldpnn_hc = pd.read_csv('../../data/charged_regions/cr_trimmed_filtered_aflabel.csv')
fldpnn_hc.rename(columns={'left.bound': 'left_bound', 'right.bound': 'right_bound', 'region.seq': 'region_seq'}, inplace=True)
fldpnn_hc['fldpnn_label'] = fldpnn_hc.apply(lambda row: append_fldpnn_label(row), axis=1)
fldpnn_hc.head(5)

,orf,gene,seq.len,left_bound,right_bound,region_seq,region.len,charge.asymmetry,frac.charge,uni_id,orf_label,kappa,label,fldpnn_label
0,YAL011W,SWC3,626,5,53,RTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDDAYDEVGND,49,0.017593,0.591837,P31376,verified,0.539053,disordered,disordered
1,YAL011W,SWC3,626,169,261,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,93,0.052936,0.559140,P31376,verified,0.098538,helix,helix
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,P31376,verified,0.089231,disordered,disordered
3,YAL013W,DEP1,406,84,159,ESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAIDSK...,76,0.138444,0.592105,P31385,verified,0.219047,disordered,disordered
4,YAL019W,FUN30,1132,463,538,ERETKRIRNTTKPKVVEDEDDDVDLEAIDDELPQSEHEDDDYEEED...,76,0.269474,0.644737,P31380,verified,0.498598,disordered,disordered


In [10]:
with open('../../data/misc/uniprot-to-sgdid.txt') as f:
    mappings = {}
    for line in f:
        uniprot = line[95:106].rstrip()
        orf = line[75:95].rstrip()
        mappings[uniprot] = orf

In [14]:
def append_orf(row):
    return mappings[row['uni_id']]

In [28]:
fldpnn_random = pd.read_csv('../../data/af_regions/random_af_regions_low_thresh.csv')
fldpnn_random['orf'] = fldpnn_random.apply(lambda row: append_orf(row), axis=1)
fldpnn_random.rename(columns={'seq': 'region_seq'}, inplace=True)
fldpnn_random['fldpnn_label'] = fldpnn_random.apply(lambda row: append_fldpnn_label(row), axis=1)
fldpnn_random.head(5)

IGIAIVFAALINGVSRNPSIKDLVFPMAILGFALSEATGLFCLM
IGIAIVFAALINGVSRNPSIKDTVFPMAILGFALSEATGLFCLM


,uni_id,region_seq,left_bound,right_bound,label,orf,fldpnn_label
0,Q04007,FYAPSILSRSGSSTDVLSSGIDSMAKNSKETRGRFRSLSLMDPALQ...,503,552,disordered,YDR186C,None
1,P22516,PIAEVVTLPYQYLLSESTRSSLQINLENSIVIIDEAHNLIETINSI...,359,452,helix,YPL008W,None
2,Q12218,APSSSEVVSSSVASSSSEVASSSVAPSSSEVVSSSVASSSSEVASS...,164,228,disordered,YOR009W,None
3,P38196,TIPTEDYEEITKESEMGDATKITSKIDANVIEKKDTDSENNITIAQ...,18,94,disordered,YBL042C,None
4,Q12186,IQGIVCKICGQTGHFSRDCNSSSQRMSRFDRNATVNNSAPIQSNDV...,293,369,disordered,YLR116W,None


In [9]:
df = pd.read_csv('../../data/af_regions/sc_af_regions_all.csv')
df.head(5)

,start,end,uni_id,region_seq,len_region,label
0,1143,1185,P39702,VQDLKQLLLNVFNTYKLERSLSELIQKIIEDSSQDLVQQYRKF,43,helix
1,508,551,P10591,KEDIEKMVAEAEKFKEEDEKESQRIASKNQLESIAYSLKNTISE,44,helix
2,138,208,P39704,DIIANNAVEEIDRNLNKITKTLNYLRAREWRNMSTVNSTESRLTWL...,71,helix
3,103,142,P18411,KQMFLGSLFGVVLGVTVAKISILFMYVGITSMLLCEWLRY,40,helix
4,45,93,P18410,PASMIFRNLLILEDDLRRQAHEQKILKWQFTLFLASMAGVGAFTFYELY,49,helix


In [6]:
df_hc.head(5)

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,uni_id,orf_label,kappa,label,fldpnn_label
0,YAL011W,SWC3,626,5,53,RTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDDAYDEVGND,49,0.017593,0.591837,P31376,verified,0.539053,disordered,disordered
1,YAL011W,SWC3,626,169,261,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,93,0.052936,0.559140,P31376,verified,0.098538,helix,helix
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,P31376,verified,0.089231,disordered,disordered
3,YAL013W,DEP1,406,84,159,ESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAIDSK...,76,0.138444,0.592105,P31385,verified,0.219047,disordered,disordered
4,YAL019W,FUN30,1132,463,538,ERETKRIRNTTKPKVVEDEDDDVDLEAIDDELPQSEHEDDDYEEED...,76,0.269474,0.644737,P31380,verified,0.498598,disordered,disordered


In [ ]:
uv_all = pd.read_csv('../data/uversky/uversky_all.csv')
uv_rd = pd.read_csv('../data/uversky/uversky_random.csv')
uv_hc = pd.read_csv('../data/uversky/uversky_hc_trimmed.csv')
df_uv = [uv_all, uv_rd, uv_hc]

In [ ]:
cnf_uv_all = np.array([[0, 0], [0, 0]])
cnf_uv_rd = np.array([[0, 0], [0, 0]])
cnf_uv_hc = np.array([[0, 0], [0, 0]])
cnf_uv = [cnf_uv_all, cnf_uv_rd, cnf_uv_hc]

In [ ]:
for i in range(3):
    df = df_uv[i]
    cnf = cnf_uv[i]
    cnf[0, 0] = len(df[(df.label == 'disordered') & (df.uversky_pred == 'disordered')])
    cnf[0, 1] = len(df[(df.label == 'disordered') & (df.uversky_pred == 'helix')])
    cnf[1, 0] = len(df[(df.label == 'helix') & (df.uversky_pred == 'disordered')])
    cnf[1, 1] = len(df[(df.label == 'helix') & (df.uversky_pred == 'helix')])

In [ ]:
sample_size = []
for df in df_uv:
    sample_size.append(len(df))

In [ ]:
sample_size

[3360, 3405, 681]

In [ ]:
def get_cnf_freq(cnaf_mat):
    cnf_freq = cnf_mat / cnf_mat.sum(axis=1)[:, np.newaxis]
    return cnf_freq

In [ ]:
cnf_matrices = [cnf_all, cnf_rd, cnf_hc] + cnf_uv

In [ ]:
cnf_df = pd.DataFrame(columns={'TN', 'FP', 'FN', 'TP', 'n'})
i = 0
for mat in cnf_matrices:
    rv = {}
    rv['TN'] = mat[0][0]
    rv['FP'] = mat[0][1]
    rv['FN'] = mat[1][0]
    rv['TP'] = mat[1][1]
    rv['n'] = sample_size[i]
    cnf_df = cnf_df.append(rv, ignore_index=True)
    i += 1
    if i == 3:
        i = 0

In [ ]:
cnf_df.to_csv('../misc/cp_cnf_matrices_posttrim.csv', index=False)